In [9]:
import pandas as pd
import os
from helpers import read_csv
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.model_selection import train_test_split
import numpy as np
from joblib import parallel_backend

DATASET_BASE = "dataset"
PICKLED_OBJS = "pickled_objs"

txs_classes:pd.DataFrame = read_csv(os.path.join(DATASET_BASE, "elliptic_txs_classes.csv"), ret_Dataframe=True)
txs_edgelist:pd.DataFrame = read_csv(os.path.join(DATASET_BASE, "elliptic_txs_edgelist.csv"), ret_Dataframe=True)
txs_features:pd.DataFrame =  read_csv(os.path.join(DATASET_BASE, "elliptic_txs_features.csv"), ret_Dataframe=True)


merge_feats_class = pd.merge(txs_features, txs_classes, on='txId').dropna()

merge_feats_class = merge_feats_class.loc[merge_feats_class['class'].isin([1, 2])].reset_index(drop=True)


merge_feats_class

Fitting estimator with 173 features.
Fitting estimator with 173 features.


,txId,Time step,Local_feature_1,Local_feature_2,Local_feature_3,Local_feature_4,Local_feature_5,Local_feature_6,Local_feature_7,Local_feature_8,...,in_BTC_max,in_BTC_mean,in_BTC_median,in_BTC_total,out_BTC_min,out_BTC_max,out_BTC_mean,out_BTC_median,out_BTC_total,class
0,68869,1,-0.114267,-0.184668,-1.201369,0.028105,-0.043875,-0.113002,0.547008,-0.161652,...,8.000000,3.102967,1.000000,9.308900,1.229000e+00,8.079800,4.654400,4.654400,9.308800,2
1,89273,1,5.202107,-0.210553,-1.756361,-0.121970,260.090707,-0.113002,-0.061584,5.335864,...,852.164680,852.164680,852.164680,852.164680,1.300000e-07,41.264036,0.065016,0.000441,852.164680,2
2,293323,1,-0.172726,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.163383,...,0.040774,0.040774,0.040774,0.040774,9.480000e-04,0.039726,0.020337,0.020337,0.040674,2
3,1494462,1,-0.172921,-0.158783,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.163581,...,0.010000,0.010000,0.010000,0.010000,2.900000e-03,0.006900,0.004900,0.004900,0.009800,2
4,1582950,1,-0.169967,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160559,...,0.478277,0.478277,0.478277,0.478277,4.630000e-02,0.431877,0.239088,0.239088,0.478177,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46040,194334585,49,-0.039416,-0.118083,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,-0.026985,...,21.175927,21.175927,21.175927,21.175927,4.005752e-02,21.135512,10.587785,10.587785,21.175569,2
46041,194334621,49,-0.050308,-0.112834,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,-0.038129,...,19.449182,19.449182,19.449182,19.449182,1.000000e-02,19.438805,9.724402,9.724402,19.448805,2
46042,194335206,49,-0.154605,-0.116753,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,-0.144840,...,2.914019,2.914019,2.914019,2.914019,3.886639e-01,2.524993,1.456828,1.456828,2.913657,2
46043,194335216,49,0.708000,-0.118083,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,0.737731,...,139.670987,139.670987,139.670987,139.670987,1.018483e-01,139.568782,69.835315,69.835315,139.670630,2


Fitting estimator with 172 features.
Fitting estimator with 172 features.


In [13]:
merge_feats_class = merge_feats_class.drop('txId',axis=1)
merge_feats_class = merge_feats_class.drop('Time step',axis=1)
merge_feats_class


,Local_feature_1,Local_feature_2,Local_feature_3,Local_feature_4,Local_feature_5,Local_feature_6,Local_feature_7,Local_feature_8,Local_feature_9,Local_feature_10,...,in_BTC_max,in_BTC_mean,in_BTC_median,in_BTC_total,out_BTC_min,out_BTC_max,out_BTC_mean,out_BTC_median,out_BTC_total,class
0,-0.114267,-0.184668,-1.201369,0.028105,-0.043875,-0.113002,0.547008,-0.161652,-0.118555,0.300047,...,8.000000,3.102967,1.000000,9.308900,1.229000e+00,8.079800,4.654400,4.654400,9.308800,2
1,5.202107,-0.210553,-1.756361,-0.121970,260.090707,-0.113002,-0.061584,5.335864,5.252974,-0.049707,...,852.164680,852.164680,852.164680,852.164680,1.300000e-07,41.264036,0.065016,0.000441,852.164680,2
2,-0.172726,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.163383,-0.169201,-0.049707,...,0.040774,0.040774,0.040774,0.040774,9.480000e-04,0.039726,0.020337,0.020337,0.040674,2
3,-0.172921,-0.158783,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.163581,-0.169397,-0.049707,...,0.010000,0.010000,0.010000,0.010000,2.900000e-03,0.006900,0.004900,0.004900,0.009800,2
4,-0.169967,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160559,-0.166417,-0.049707,...,0.478277,0.478277,0.478277,0.478277,4.630000e-02,0.431877,0.239088,0.239088,0.478177,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46040,-0.039416,-0.118083,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,-0.026985,-0.034715,-0.049707,...,21.175927,21.175927,21.175927,21.175927,4.005752e-02,21.135512,10.587785,10.587785,21.175569,2
46041,-0.050308,-0.112834,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,-0.038129,-0.045703,-0.049707,...,19.449182,19.449182,19.449182,19.449182,1.000000e-02,19.438805,9.724402,9.724402,19.448805,2
46042,-0.154605,-0.116753,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,-0.144840,-0.150918,-0.049707,...,2.914019,2.914019,2.914019,2.914019,3.886639e-01,2.524993,1.456828,1.456828,2.913657,2
46043,0.708000,-0.118083,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,0.737731,0.719284,-0.049707,...,139.670987,139.670987,139.670987,139.670987,1.018483e-01,139.568782,69.835315,69.835315,139.670630,2


In [4]:

# Encode the class labels


# Separate features and target
X = merge_feats_class.drop('class', axis=1)
y = merge_feats_class['class']

# Split the data into a training set and a validation set
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

# Initialize a Random Forest classifier (you can use your preferred classifier)
clf = RandomForestClassifier(n_estimators=50, random_state=42)

# Initialize RFECV with cross-validation (change cv value as needed)
rfecv = RFECV(estimator=clf, step=1, cv=2, scoring='accuracy', verbose=1)

with parallel_backend("threading"):
# Fit RFECV to your training data
    rfecv.fit(X_train, y_train)

# Get the selected feature indices
selected_feature_indices = rfecv.support_

KeyError: "['txId', 'Time step'] not found in axis"

Fitting estimator with 180 features.
Fitting estimator with 180 features.
Fitting estimator with 179 features.
Fitting estimator with 179 features.
Fitting estimator with 178 features.


In [ ]:
# print(specific_timestep.columns[np.where(selected_feature_indices)])
# print(rfecv.estimator_.feature_importances_)
feature_names = list(specific_timestep.columns[np.where(selected_feature_indices)].values)
importances = list(rfecv.estimator_.feature_importances_)
print(len(feature_names))

feature_df = pd.DataFrame([importances], columns=feature_names)
from helpers import pickle_data
#pickle_data("feature_importances.pkl", feature_df.iloc[0].to_dict())
#print(feature_df)



29


IndexError: index 40 is out of bounds for axis 0 with size 29